In [1]:
#!pip install backtrader

In [14]:
!conda install -c conda-forge matplotlib=3.2.2 -y

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done

## Package Plan ##

  environment location: /home/manuj/anaconda3/envs/quant

  added / updated specs:
    - matplotlib=3.2.2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    matplotlib-base-3.2.2      |   py38hef1b27d_0         5.4 MB
    ------------------------------------------------------------
                                           Total:         5.4 MB

The following NEW packages will be INSTALLED:

  python_abi         conda-forge/linux-64::python_abi-3.8-1_cp38

The following packages will be UPDATED:

  certifi            pkgs/main::certifi-2020.12.5-py38h06a~ --> conda-forge::certifi-2020.12.5-py38h578d9bd_1

The following packages will be SUPERSEDED by a 

In [2]:
rom datetime import datetime
import backtrader as bt
import matplotlib

matplotlib.__version__

'3.2.2'

In [3]:
cerebro = bt.Cerebro()

In [4]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    
    params = (
        ('exitbars', 5),
        ('maperiod', 15)
    )

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        
        # To keep track of pending orders
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.maperiod)

        # Indicators for the plotting show
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25,
                                            subplot=True)
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi, period=10)
        bt.indicators.ATR(self.datas[0], plot=False)

    def log(self, txt, dt=None):
        '''Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()}, {txt}')
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
        
        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, Price: {order.executed.price}, Cost: {order.executed.value}, Comm {order.executed.comm}')
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm 
            elif order.issell():
                self.log(f'SELL EXECUTED, Price: {order.executed.price}, Cost: {order.executed.value}, Comm {order.executed.comm}')
    
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Reset order status to prepare for next order
        self.order = None
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log(f'OPERATION PROFIT, GROSS: {trade.pnl}, NET: {trade.pnlcomm}')

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log(f'Close, {self.dataclose[0]}')

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] < self.dataclose[-1]:
                # current close less than previous close

                if self.dataclose[-1] < self.dataclose[-2]:
                    # previous close less than the previous close

                    # BUY, BUY, BUY!!! (with default parameters)
                    self.log(f'BUY CREATE, {self.dataclose[0]}')

                    # Keep track of the created order to avoid a 2nd order
                    self.order = self.buy()

        else:
            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + self.params.exitbars):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log(f'SELL CREATE, {self.dataclose[0]}')

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [5]:
# Add a strategy
cerebro.addstrategy(TestStrategy)

0

In [6]:
data = bt.feeds.YahooFinanceData(dataname='^NSEI', fromdate=datetime(2020, 1, 1),
                                  todate=datetime(2021, 2, 16))
cerebro.adddata(data)

In [7]:
cerebro.broker.setcash(10_00_000.0)

In [8]:
# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=3) #buy/sell 3 futures

In [9]:
# Set the commission - Rs 20 per transcation
cerebro.broker.setcommission(commission=20, margin=1_70_000)

In [10]:
starting = cerebro.broker.getvalue()
print(f'Starting Portfolio Value: {starting}')

Starting Portfolio Value: 1000000.0


In [11]:
%%time
cerebro.run()

2020-02-17, Close, 12045.8
2020-02-17, BUY CREATE, 12045.8
2020-02-18, BUY EXECUTED, Price: 12028.25, Cost: 510000.0, Comm 60.0
2020-02-18, Close, 11992.5
2020-02-19, Close, 12125.9
2020-02-20, Close, 12080.85
2020-02-24, Close, 11829.4
2020-02-25, Close, 11797.9
2020-02-26, Close, 11678.5
2020-02-26, SELL CREATE, 11678.5
2020-02-27, SELL EXECUTED, Price: 11661.25, Cost: 510000.0, Comm 60.0
2020-02-27, OPERATION PROFIT, GROSS: -1101.0, NET: -1221.0
2020-02-27, Close, 11633.3
2020-02-27, BUY CREATE, 11633.3
2020-02-28, BUY EXECUTED, Price: 11382.0, Cost: 510000.0, Comm 60.0
2020-02-28, Close, 11201.75
2020-03-02, Close, 11132.75
2020-03-03, Close, 11303.3
2020-03-04, Close, 11251.0
2020-03-05, Close, 11269.0
2020-03-06, Close, 10989.45
2020-03-06, SELL CREATE, 10989.45
2020-03-09, SELL EXECUTED, Price: 10742.05, Cost: 510000.0, Comm 60.0
2020-03-09, OPERATION PROFIT, GROSS: -1919.8500000000022, NET: -2039.8500000000022
2020-03-09, Close, 10451.45
2020-03-09, BUY CREATE, 10451.45
2020-03

In [12]:
ending = cerebro.broker.getvalue()
print(f'Final Portfolio Value: {ending}')

Final Portfolio Value: 998555.6499999997


In [13]:
print(f'Returns: {round(((ending - starting)/starting)*100, 2)}%')

Returns: -0.14%


In [14]:
cerebro.plot()

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 8 Axes>]]